In [ ]:
!pip install gradio numpy torch transformers librosa soundfile tqdm googletrans==4.0.0-rc1

In [ ]:
from googletrans import Translator
import gradio as gr
import numpy as np
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
import os
import tempfile
import soundfile as sf
from tqdm import tqdm
import shutil
import re

SAMPLING_RATE = 16000

model_name = 'ivrit-ai/whisper-large-v2-tuned'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)
processor = WhisperProcessor.from_pretrained(model_name)
translator = Translator()

def find_silent_sections(audio_data, sr, min_silence_length=0.5, silence_threshold=-40):
    silent_sections = librosa.effects.split(audio_data, top_db=-silence_threshold)
    silent_sections = [s for s in silent_sections if (s[1] - s[0]) >= min_silence_length * sr]
    return silent_sections

def split_into_paragraphs(text, min_words_per_paragraph=20):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    paragraphs = []
    current_paragraph = ""
    for sentence in sentences:
        current_paragraph += sentence + " "
        if len(current_paragraph.split()) >= min_words_per_paragraph:
            paragraphs.append(current_paragraph.strip())
            current_paragraph = ""
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())
    return paragraphs

def transcribe_and_translate(audio_file, target_language):
    audio, rate = librosa.load(audio_file, sr=None)
    if rate != SAMPLING_RATE:
        audio = librosa.resample(audio, orig_sr=rate, target_sr=SAMPLING_RATE)
    temp_dir = tempfile.mkdtemp()
    chunks = np.array_split(audio, indices_or_sections=int(np.ceil(len(audio) / (SAMPLING_RATE * 30))))
    transcribed_text = ""
    for i, chunk in enumerate(chunks):
        chunk_path = os.path.join(temp_dir, f"chunk_{i}.wav")
        sf.write(chunk_path, chunk, samplerate=SAMPLING_RATE)
        chunk_audio, _ = librosa.load(chunk_path, sr=SAMPLING_RATE)
        input_features = processor(chunk_audio, sampling_rate=SAMPLING_RATE, return_tensors="pt").input_features.to(device)
        predicted_ids = model.generate(input_features, num_beams=5)
        chunk_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        transcribed_text += chunk_text + " "
    shutil.rmtree(temp_dir)
    detected_language = translator.detect(transcribed_text).lang
    if detected_language == 'he' and target_language != 'Hebrew':
        transcribed_text = translator.translate(transcribed_text, src='he', dest=target_language.lower()).text
    return transcribed_text

title = 'Unlimited Length Transcription and Translation'
description = 'This tool transcribes audio files and translates the transcription into the selected target language. It uses Whisper for transcription and Google Translate for translations involving Hebrew.'

interface = gr.Interface(
    fn=transcribe_and_translate,
    inputs=[gr.Audio(type='filepath', label='Upload Audio File'), gr.Dropdown(choices=['Hebrew', 'English', 'Spanish', 'French'], label='Target Language')],
    outputs=gr.Textbox(label='Transcription / Translation'),
    title=title,
    description=description
)

if __name__ == '__main__':
    interface.launch()